In [45]:
from itertools import product
import torch as th
import numpy as np
import sympy as sp
from IPython.display import display, Math

In [79]:
def get_qubit_probe_states(num_qubits: int, return_dm: bool = False, device: str = 'cpu'):
    """
    Generate all probe states for `num_qubits` using the standard basis |0>, |1>, |+>, |+i>.

    Args:
        num_qubits (int): Number of qubits.
        return_dm (bool): If True, return density matrices instead of kets.
        device (str): Torch device.

    Returns:
        torch.Tensor: Tensor of shape (4**num_qubits, 2**num_qubits) for kets,
                      or (4**num_qubits, 2**num_qubits, 2**num_qubits) for density matrices.
    """
    # single-qubit states
    zero = th.tensor([1.0, 0.0], dtype=th.complex64, device=device)
    one = th.tensor([0.0, 1.0], dtype=th.complex64, device=device)
    plus = (zero + one) / th.sqrt(th.tensor(2.0))
    plus_i = (zero + 1j * one) / th.sqrt(th.tensor(2.0))
    
    basis = [zero, one, plus, plus_i]
    
    # generate all combinations of basis states for num_qubits
    states = []
    for combo in product(basis, repeat=num_qubits):
        ket = combo[0]
        for b in combo[1:]:
            ket = th.kron(ket, b)
        if return_dm:
            rho = ket.unsqueeze(-1) @ ket.conj().unsqueeze(0)
            states.append(rho)
        else:
            states.append(ket)
    
    return th.stack(states)



def compute_expectation_value(state: th.Tensor, operator: th.Tensor) -> float:
    """
    Compute the expectation value of an operator given a quantum state.
    
    Args:
        state (Tensor): Either a ket |psi> (shape [dim]) or a density matrix rho (shape [dim, dim])
        operator (Tensor): Hermitian operator E (shape [dim, dim])
    
    Returns:
        float: expectation value <E>
    """
    assert operator.dim() == 2, "Operator must have 2 dimensions."
    
    if state.dim() == 1:  # ket
        # <psi|E|psi>
        exp_val = th.conj(state) @ operator @ state
    elif state.dim() == 2:  # density matrix
        # Tr[rho*E]
        exp_val = th.trace(state @ operator)
    else:
        raise ValueError(f"State must be 1D (ket) or 2D (density matrix), got shape {state.shape}")
    
    return exp_val.real.item()


In [80]:
probes = get_qubit_probe_states(num_qubits=2, return_dm=not True)
print(f"Hilbert dimension: {probes[0].shape[0]}")

Hilbert dimension: 4


In [81]:
print(compute_expectation_value(probes[1], th.outer(probes[2].conj(), probes[2]) ))

0.4999999701976776


In [82]:
if probes[0].shape[0] < 5:
    for i, A in enumerate(probes.numpy()):
        if len(probes[0].shape) > 1:
            display(Math(rf"\rho_{{\scriptstyle{i+1}}} = {sp.latex(sp.Matrix(np.round(A, 3)))}"))
        else:
            display(Math(rf"\psi_{{\scriptstyle{i+1}}} = {sp.latex(sp.Matrix(np.round(A, 3)))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>